In [1]:
%cd /Users/kimmonica/Documents/GitHub/py_storage
from basic_preprocessing import *
from deeplearning_check import *
from machine_learning import *
from FINAL_IMG_PREP import *
from video_info import *

gpu_check()
set_global_determinism(seed=SEED)

%cd /Users/kimmonica/Documents/Smart_CCTV/motion_detector

/Users/kimmonica/Documents/GitHub/py_storage
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 589018800395885397
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 2966340257369758456
physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
xla_global_id: -1
]Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB


tf 2.9.2
keras 2.9.0
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라
/Users/kimmonica/Documents/Smart_CCTV/motion_detector


2022-10-12 21:36:45.401104: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-12 21:36:45.401227: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
# import tensorflow as tf
import tensorflow_hub as hub

# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

model =hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

2022-10-12 21:36:49.508563: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-12 21:36:49.508585: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)
            
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)
            
#function to loop through each person detected and render

def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)


In [4]:
folder_path = '/Users/kimmonica/Documents/Smart_CCTV/dataset/4.쓰러짐/'

list_ = os.listdir(folder_path)
file_list=[]
for i in list_:
    if '.mp4'in i:
        file_list.append(i)
       
file_list[:3]

['trim_C052303_006.mp4', 'trim_C052303_012.mp4', 'trim_C058203_001.mp4']

### read vid

In [10]:
path = '/Users/kimmonica/Documents/Smart_CCTV/dataset/test.mp4'
cap, width, height, fps, fourcc, filename, out = vid_info(path, 'XVID','1011_test')

< cv2.VideoCapture 0x2d0d8f5b0>
cap < cv2.VideoCapture 0x2d0d8f5b0>, width 1280.0, height 720.0, fps 30.000068337285505
codec {0} filename 1011_test.mp4


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [16]:
path = '/Users/kimmonica/Documents/Smart_CCTV/dataset/test.mp4'

cap = cv2.VideoCapture(path)

fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('test_key.mp4', fourcc, 30, (600, 300))

if not cap.isOpened():
    print('Video open failed!')
    sys.exit()

ret, org_frame = cap.read()
if not ret:
    print('Background image registration failed!')
    sys.exit()

while True:
    blank_img = np.zeros((600, 300, 3), np.uint8)
    # show(blank_img)
    ret, org_frame = cap.read()
    
    if not ret:
        break
    ####### keypoint ######
    frame = org_frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # show(frame)
    img = org_frame.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    confidence_threshold = 0.3
    y, x, c = frame.shape
                # show(img)
                ## 사이즈 모델이 32의 배수여야하고 256pixel이상이여야함
                ### ratio 따라서 reshape해야함 
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 352,608)
    input_img = tf.cast(img, dtype=tf.int32)
            #
                # Detection section
    results = movenet(input_img)

                #rendering
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))

    # print(len(keypoints_with_scores))
    for i in keypoints_with_scores:
        shaped = np.squeeze(np.multiply(i, [y,x,1]))
        ky_list = []
        kx_list = []
        result_list = []
        for kp in shaped: ## 사람 머리부터 다 나옴 
            ky, kx, kp_conf = kp
            # print(len(kp))
            if kp_conf > confidence_threshold:
                ky_list.append(int(ky))
                kx_list.append(int(kx))
                loop_through_people(blank_img, keypoints_with_scores, EDGES, 0.3)
    cv2.imshow('frame',blank_img)
    out.write(blank_img)
                # draw_img = frame.copy()
                # # print(draw_img.shape)
                # draw_img = cv2.rectangle(draw_img,((min(kx_list)-50, min(ky_list)-70)), (((max(kx_list)+70, max(ky_list)+70))), (255,255,0),3 )
            
        # try:
        #     trim_org = blank_img.copy()
        #     trim=trim_org[min(ky_list)-50:max(ky_list)+70,min(kx_list)-50:max(kx_list)+70 ]
        #     trim = cv2.cvtColor(trim, cv2.COLOR_BGR2GRAY)
        #     trim = cv2.cvtColor(trim, cv2.COLOR_GRAY2RGB)
        #     trim = cv2.resize(trim,(230,600))
        #     show(trim)
        #     # cv2.imshow('frame', trim)
        # except:
        #     pass        


    if cv2.waitKey(30) == 27:
            break


cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


-1

In [6]:
/pwd

'/Users/kimmonica/Documents/Smart_CCTV/motion_detector'